# 分类器性能评估指标和方法

1. 准确率通常无法成为分类器的首要性能指标, 特别是当你处理偏态分布(skewed dataset)的时候(即某些类比其他类的样本更多)；
2. 评估分类器性能更好的方法是混淆矩阵(confusion matrix)；

## 1.混淆矩阵

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

* 混淆矩阵中的行表示**实际类别**, 列表示**预测类别**；
* 一个完美的分类器只有**真负类(TP, true positive)**和**真正类(TN, true negtive)**, 即混淆矩阵只会在对角线上有非零值；
* 混淆矩阵能提供大量分类信息, 但有时可能希望指标更简洁一些: 
    - 正类预测的准确率, 即分类器的精度

## 1.2 精度 precision

$$precision = \frac{TP}{TP + FP}$$

其中: 

* TP是正真类的数量, 混淆矩阵的右下角；
* FP是假正类的数量, 混淆矩阵的右上角；
* $100\%$是完美的精度, 但是没有什么意义, 因为分类器会忽略这个正类实例之外的所有内容；
* 精度(precision)通常与召回率(recall, 灵敏度(sensitivity))一起使用, ；

## 1.3 召回率 recall

$$recall = \frac{TP}{TP + FN}$$

其中: 

* TP是正真类的数量, 混淆矩阵的右下角；
* FN是假负类的数量, 混淆矩阵的左下角；

### 精度, 召回率实现

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_true, y_pred)
recall_score(y_true, y_pred)

### 精度, 召回率解释

* 精度:
    - 当判断一个样本是正例时, 有多少的样本是预测准确的；
* 召回率:
    - 当谈论一个样本是正例时, 只有多少正真的正例被预测正确；

## 1.4 F1 Score

$$F1 = \frac{2}{\frac{1}{precision} + \frac{1}{recall}}$$

$$F1 = \frac{TP}{TP + \frac{FN+FP}{2}}$$

* F1 score是精度和召回率的调和平均值；
* 当需要一个更简单的方法比较两个分类器的性能时, F1 score是一个非常不错的指标；

### F1 score的实现

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_true, y_pred)

### F1 score解释

* F1 score对于那些具有相近的精度和召回率的分类器更有用；

## 1.5 精度/召回率权衡

#### 为什么要权衡

在某些情况下, 更关心的是精度, 而另一些情况下, 更关心的是召回率；

实际分类情况举例: 

1. 对于把有病的病人检测检测出来
    - 将很多健康(负例)的病人检测为有病(高召回率), 但对真正有病的病人(真例)检测出来的都为有病(低精度)的分类器是好的分类器；
2. 使用分类器检测儿童可以放心看的视屏
    - 拦截了很多好视频(真例)(低召回率), 但保留下来的视屏都是安全的(高精度)的分类器是好的分类器；
3. 使用分类器监控检测小偷
    - 安保人员可能会收到一些错误的警报(高召回率), 但是几乎所有的窃贼都在劫难逃(低精度)；

#### 权衡的原理

* 很多分类器都是基于决策函数计算出一个分值, 如果该值大于阈值, 则将该实例判为正, 否则便将其判为负；不同的阈值分类的结果就会不同, 得到的精度和召回率就会不一样；
* sklearn不允许直接设置阈值, 但是可以范围它用于预测的决策分数:
    - `decision_function()`:返回每个样本实例的决策分数, 可以根据这些分数, 使用任意阈值进行预测；

In [7]:
import numpy as np

threshold = np.linspace(0.0, 1.0, 10)
# y_score = clf.decision_function(X_train)
y_score = np.linspace(0.0, 1.0, 20)

y_pred = dict()
for t in threshold:
    y_pred[t] = (y_score > t)

print(y_pred)

{0.0: array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True], dtype=bool), 0.1111111111111111: array([False, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True], dtype=bool), 0.22222222222222221: array([False, False, False, False, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True], dtype=bool), 0.33333333333333331: array([False, False, False, False, False, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True], dtype=bool), 0.44444444444444442: array([False, False, False, False, False, False, False, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True], dtype=bool), 0.55555555555555558: array([False, F

### 如何决定使用什么阈值？

#### 1.精度, 召回率相对于阈值的函数

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_recall_curve

# 获得训练集中所有样本实例的分数
y_score = cross_val_predict(clf, X_train, y_train, 
                            cv = 5, 
                            method = "decision_function")

# 计算所有可能的阈值的精度和召回率
precisions, recalls, thresholds = precision_recall_curve(y_true, y_score)

# 回执精度和召回率相对于阈值的函数图
def plot_precision_recall_vs_threshold(precision, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label = "Precision")
    plt.plot(thresholds, recalls[:-1], "g--", label = "Recall")
    plt.xlabel("Thresholds")
    plt.legend(loc = "upper left")
    plt.ylim([0, 1])
    plt.show()
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)

#### 2.精度和召回率函数

In [ ]:
plot(recall, precision)

# 2.ROC、AUC

## 2.1 ROC

* ROC曲线绘制的是灵敏度和(1-特异度)的关系
    - 灵敏度: $真正类率(TPR)/召回率 = \frac{TP}{TP + FN}$
    - 特异度: $真负类率 = \frac{TN}{TN + FP}$
        - 被正确分类为负类的负类实例比率
        - 假正类率: 被错误分为正类的负类实例的比率；
            - $假正类率(FPR) = 1 - 真负类率 = \frac{FP}{FP + TN}$

### 绘制ROC曲线

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train, y_scores)

def plot_roc_curve(fpr, tpr, label = None):
    plt.plot(fpr, tpr, linewidth = 2, label = label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")

plot_roc_curve(fpr, tpr)
plt.show()

### AUC

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train, y_scores)